<h1>导入数据</h1>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import time
import tensorflow.keras.backend as K
import keras_metrics as km
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from tensorflow.keras.layers import Input, Dense, LSTM, MaxPool1D, Dropout, concatenate, Flatten, Conv1D, Lambda, BatchNormalization, ReLU, Activation
from tensorflow.keras.callbacks import History, TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import Hinge
from tensorflow.keras.metrics import Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.backend import clear_session, epsilon
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, save_model, load_model
from time import strftime, localtime
from itertools import combinations, product

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [2]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [3]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [4]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [5]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [6]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [7]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [8]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [9]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [10]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0,0,0,0,0,0,1,0,1,1


In [11]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                8840 non-null   float64
 1   RUN_CAP                     8840 non-null   float64
 2   SHIFT_NO                    8840 non-null   float64
 3   CHK_CYCLE                   8840 non-null   float64
 4   IS_FLAG                     8840 non-null   int64  
 5   live_days                   8840 non-null   int64  
 6   check_days                  8840 non-null   int64  
 7   ELEC_TYPE_NAME_乡村居民生活用电     8840 non-null   uint8  
 8   ELEC_TYPE_NAME_商业用电         8840 non-null   uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     8840 non-null   uint8  
 10  ELEC_TYPE_NAME_学校教学和学生生活用电  8840 non-null   uint8  
 11  ELEC_TYPE_NAME_居民生活用电       8840 non-null   uint8  
 12  ELEC_TYPE_NAME_普通工业         8840 non-null   uint8  
 13  ELEC_TYPE_NAME_非居民照

<h3>elec_month数据预处理</h3>

In [12]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [13]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] = elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] / elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]].max(axis=0)
elec_month

In [14]:
cons_month_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_month_result.append(df.values)
cons_month = np.array(cons_month_result)
cons_month.shape

100%|███████████████████████████████████████████████████████████████████████████████| 8840/8840 [00:14<00:00, 610.12it/s]


(8840, 22, 4)

<h3>elec_day数据预处理</h3>

In [15]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] = elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] / elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]].max(axis=0)
elec_day

In [16]:
dates = pd.to_datetime(elec_day["rq"].unique()).sort_values()

In [17]:
cons_day_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id].drop(columns=["id"]).sort_values("rq")
    df["rq"] = pd.to_datetime(df["rq"])
    df = df.set_index("rq").reindex(dates)
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_day_result.append(df.values)
cons_day = np.array(cons_day_result)
cons_day.shape

100%|███████████████████████████████████████████████████████████████████████████████| 8840/8840 [00:29<00:00, 303.13it/s]


(8840, 108, 6)

<h2>再处理数据</h2>

<h3>查看现有数据形状</h3>

<h4>标签</h4>

In [18]:
target = cons_info["IS_FLAG"].values.astype(np.float64)
target.shape

(8840,)

<h4>统一视图</h4>

In [19]:
cons_info_train = cons_info.drop(columns=["IS_FLAG"]).values
print(cons_info_train.shape)
print(np.isnan(cons_info_train).sum())

(8840, 20)
0


<h4>月均电量</h4>

In [20]:
print(cons_month.shape)
print(np.isnan(cons_month).sum())

(8840, 22, 4)
0


<h4>日电量</h4>

In [21]:
print(cons_day.shape)
print(np.isnan(cons_day).sum())

(8840, 108, 6)
0


<h3>处理样本平衡性</h3>

<h4>构造黑样本</h4>

In [22]:
pos = target == 1
neg = target == 0

In [23]:
cons_info_pos = cons_info_train[pos]
cons_month_pos = cons_month[pos]
cons_day_pos = cons_day[pos]

In [24]:
target_pos_new = []
cons_info_pos_new = []
cons_month_pos_new = []
cons_day_pos_new = []
for h in tqdm(range(1, 9)):
    for i, j, k in product(list(range(10)), repeat=3):
        target_pos_new.append(1)
        cons_info_pos_new.append(cons_info_pos[h])
        cons_month_pos_new.append((1 / 4) * (cons_month_pos[h] + cons_month_pos[i] + cons_month_pos[j] + cons_month_pos[k]))
        cons_day_pos_new.append((1 / 4) * (cons_day_pos[h] + cons_day_pos[i] + cons_day_pos[j] + cons_day_pos[k]))
target_pos_new = np.array(target_pos_new)
cons_info_pos_new = np.array(cons_info_pos_new)
cons_month_pos_new = np.array(cons_month_pos_new)
cons_day_pos_new = np.array(cons_day_pos_new)


print(target_pos_new.shape)
print(cons_info_pos_new.shape)
print(cons_month_pos_new.shape)
print(cons_day_pos_new.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 80.81it/s]

(8000,)
(8000, 20)
(8000, 22, 4)
(8000, 108, 6)


<h4>合并负样本</h4>

target = np.concatenate([target_pos_new, target[neg]], axis=0)
target.shape

cons_info_train = np.concatenate([cons_info_pos_new, cons_info_train[neg]], axis=0)
cons_info_train.shape

cons_month = np.concatenate([cons_month_pos_new, cons_month[neg]], axis=0)
cons_month.shape

cons_day = np.concatenate([cons_day_pos_new, cons_day[neg]], axis=0)
cons_day.shape

<h4>打乱顺序</h4>

In [25]:
random_order = list(range(target.shape[0]))
np.random.shuffle(random_order)
len(random_order)

8840

In [26]:
target = target[random_order]
cons_info_train = cons_info_train[random_order]
cons_month = cons_month[random_order]
cons_day = cons_day[random_order]

<h1>构建模型</h1>

<h2>初始化tensorflow</h2>

In [27]:
clear_session()
tf.compat.v1.reset_default_graph()
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h2>构建模型</h2>

In [28]:
def add_features(x, columns: [int]):
    """
    增加新的特征\n
    """
    if len(columns) == 0:
        return x
    for i, j in combinations(columns, r=2):
        feature = tf.expand_dims(x[:, :, i] - x[:, :, j], -1)
        x = tf.concat([x, feature], axis=-1)
    return x

In [29]:
def build_model(cons_info_shape, cons_month_shape, cons_day_shape) -> Model:
    input_cons_info = Input(shape=cons_info_shape, name="input_cons_info")
    input_cons_month = Input(shape=cons_month_shape, name="input_cons_month")
    input_cons_day = Input(shape=cons_day_shape, name="input_cons_day")
    
    x0 = BatchNormalization(name="x0")(input_cons_info)
    x1 = Dense(units=32, activation="relu", name="x1")(x0)
    x2 = Dense(units=64, activation="relu", name="x2")(x1)
    x3 = BatchNormalization(name="x4")(x2)
    x4 = ReLU(name="x5")(x3)
    
    y0 = BatchNormalization(name="y0", axis=-1)(input_cons_month)
    y1 = Lambda(lambda x: add_features(x, [0, 1, 2]), name="y1")(y0)
    y2, y_hidden_state, y_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="y2")(y1)
    y_hidden_state_2 = Dense(units=64, name="y_hidden_state_2")(y_hidden_state)
    y_cell_state_2 = Dense(units=64, name="y_cell_state_2")(y_cell_state)
    y3 = LSTM(units=64, return_sequences=False, return_state=False, name="y3")(y2, [y_hidden_state_2, y_cell_state_2])
    
    z0 = BatchNormalization(name="z0", axis=-1)(input_cons_day)
    z1 = Lambda(lambda x: add_features(x, [3, 4, 5]), name="z1")(z0)
    z2, z_hidden_state, z_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="z2")(z1)
    z_hidden_state_2 = Dense(units=64, name="z_hidden_state_2")(z_hidden_state)
    z_cell_state_2 = Dense(units=64, name="z_cell_state_2")(z_cell_state)
    z3 = LSTM(units=64, return_sequences=False, return_state=False, name="z3")(z2, [z_hidden_state_2, z_cell_state_2])
    
    o1 = concatenate([x4, y3, z3], axis=-1, name="o1")
    o2 = ReLU(name="relu_o2")(BatchNormalization(name="bn_o2")(Dense(units=64, activation=None, name="o2")(o1)))
    o3 = ReLU(name="relu_o3")(BatchNormalization(name="bn_o3")(Dense(units=128, activation=None, name="o3")(o2)))
    o4 = ReLU(name="relu_o4")(BatchNormalization(name="bn_o4")(Dense(units=64, activation=None, name="o4")(o3)))
    o5 = ReLU(name="relu_o5")(BatchNormalization(name="bn_o5")(Dense(units=32, activation=None, name="o5")(o4)))
    o6 = ReLU(name="relu_o6")(BatchNormalization(name="bn_o6")(Dense(units=16, activation=None, name="o6")(o5)))
    o7 = Activation(activation="sigmoid", name="relu_o7")(BatchNormalization(name="bn_o7")(Dense(units=1, activation=None, name="o7")(o6)))
    
    result = Model(inputs=[input_cons_info, input_cons_month, input_cons_day], outputs=[o7], name="mining-detector")
    result.summary()
    return result

In [30]:
def f1_loss(y_true, y_pred):
    #计算tp、tn、fp、fn
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    
    #percision与recall，这里的K.epsilon代表一个小正数，用来避免分母为零
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    #计算f1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)#其实就是把nan换成0
    return 1 - K.mean(f1)

In [31]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [32]:
model = build_model(cons_info_train.shape[1:], cons_month.shape[1:], cons_day.shape[1:])
model.compile(
    optimizer=Adam(learning_rate=1e-3), 
    loss="binary_crossentropy", 
    metrics=[
        "accuracy", 
        F1Score(num_classes=1, average='weighted',threshold=0.5), 
        Precision(thresholds=0.5), 
        Recall(thresholds=0.5),
        TruePositives(thresholds=0.5),
        TrueNegatives(thresholds=0.5),
        FalsePositives(thresholds=0.5),
        FalseNegatives(thresholds=0.5)
    ]
)

Model: "mining-detector"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cons_info (InputLayer)    [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_cons_month (InputLayer)   [(None, 22, 4)]      0                                            
__________________________________________________________________________________________________
input_cons_day (InputLayer)     [(None, 108, 6)]     0                                            
__________________________________________________________________________________________________
x0 (BatchNormalization)         (None, 20)           80          input_cons_info[0][0]            
____________________________________________________________________________________

<h2>训练模型</h2>

<h3>设置class_weight和sample_weight</h3>

In [33]:
class_weight = compute_class_weight('balanced', np.unique(target) ,target)
    
sample_weights = np.ones(shape=(len(target),))
sample_weights[target == 1] = 800

c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


<h3>开始训练</h3>

In [34]:
model.fit(
    [cons_info_train, cons_month, cons_day], 
    target, 
#     validation_split=0.2,
    epochs=100, 
    batch_size=1024, 
#     class_weight=dict(enumerate(class_weight)),
    use_multiprocessing=True, 
    workers=12,
    callbacks=[
        TensorBoard(log_dir="../logs/%s" % strftime("%Y%m%d_%H.%M.%S", localtime()), write_graph=True, write_images=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
        ModelCheckpoint(filepath="../models/mining-detector-checkpoint.h5", monitor="loss", save_best_only=True, save_weights_only=True, verbose=1),
#         EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, restore_best_weights=True)
    ]
)

Epoch 1/100
1/9 [==>...........................] - ETA: 0s - loss: 0.7532 - accuracy: 0.3838 - f1_score: 0.0032 - precision: 0.0016 - recall: 0.5000 - true_positives: 1.0000 - true_negatives: 392.0000 - false_positives: 630.0000 - false_negatives: 1.0000WARNING:tensorflow:From c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
8/9 [=========================>....] - ETA: 0s - loss: 0.7363 - accuracy: 0.4261 - f1_score: 0.0017 - precision: 8.5106e-04 - recall: 0.4444 - true_positives: 4.0000 - true_negatives: 3487.0000 - false_positives: 4696.0000 - false_negatives: 5.0000WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negativ


Epoch 00009: loss improved from 0.67847 to 0.67377, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0s 40ms/step - loss: 0.6738 - accuracy: 0.7752 - f1_score: 0.0100 - precision: 0.0050 - recall: 1.0000 - true_positives: 10.0000 - true_negatives: 6843.0000 - false_positives: 1987.0000 - false_negatives: 0.0000e+00
Epoch 10/100
9/9 [==============================] - ETA: 0s - loss: 0.6728 - accuracy: 0.7342 - f1_score: 0.0084 - precision: 0.0042 - recall: 1.0000 - true_positives: 10.0000 - true_negatives: 6480.0000 - false_positives: 2350.0000 - false_negatives: 0.0000e+00WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00010: loss improved from 0.67377 to 0.67277, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0

Epoch 29/100
9/9 [==============================] - ETA: 0s - loss: 0.5832 - accuracy: 0.9843 - f1_score: 0.1146 - precision: 0.0612 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8692.0000 - false_positives: 138.0000 - false_negatives: 1.0000               WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00029: loss improved from 0.59148 to 0.58316, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0s 44ms/step - loss: 0.5832 - accuracy: 0.9843 - f1_score: 0.1146 - precision: 0.0612 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8692.0000 - false_positives: 138.0000 - false_negatives: 1.0000
Epoch 30/100
9/9 [==============================] - ETA: 0s - loss: 0.5842 - accuracy: 0.9934 - f1_score: 0.1471 - precision: 0.0862 - recall: 0.5

Epoch 39/100
9/9 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.9984 - f1_score: 0.5625 - precision: 0.4091 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8817.0000 - false_positives: 13.0000 - false_negatives: 1.0000  WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00039: loss improved from 0.54915 to 0.54149, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0s 42ms/step - loss: 0.5415 - accuracy: 0.9984 - f1_score: 0.5625 - precision: 0.4091 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8817.0000 - false_positives: 13.0000 - false_negatives: 1.0000
Epoch 40/100
9/9 [==============================] - ETA: 0s - loss: 0.5384 - accuracy: 0.9983 - f1_score: 0.5455 - precision: 0.3913 - recall: 0.9000 - true_posi

Epoch 49/100
9/9 [==============================] - ETA: 0s - loss: 0.5164 - accuracy: 0.9752 - f1_score: 0.0681 - precision: 0.0356 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8613.0000 - false_positives: 217.0000 - false_negatives: 2.0000                WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00049: loss improved from 0.51659 to 0.51641, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0s 39ms/step - loss: 0.5164 - accuracy: 0.9752 - f1_score: 0.0681 - precision: 0.0356 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8613.0000 - false_positives: 217.0000 - false_negatives: 2.0000
Epoch 50/100
9/9 [==============================] - ETA: 0s - loss: 0.5110 - accuracy: 0.9755 - f1_score: 0.0844 - precision: 0.0441 - recall: 1.

Epoch 59/100
9/9 [==============================] - ETA: 0s - loss: 0.4857 - accuracy: 0.9971 - f1_score: 0.3810 - precision: 0.2500 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8806.0000 - false_positives: 24.0000 - false_negatives: 2.0000              WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00059: loss did not improve from 0.48434
9/9 [==============================] - 0s 31ms/step - loss: 0.4857 - accuracy: 0.9971 - f1_score: 0.3810 - precision: 0.2500 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8806.0000 - false_positives: 24.0000 - false_negatives: 2.0000
Epoch 60/100
9/9 [==============================] - ETA: 0s - loss: 0.4761 - accuracy: 0.9977 - f1_score: 0.5000 - precision: 0.3333 - recall: 1.0000 - true_positives: 10.0000 - true_negatives: 8810.0000 - fals

Epoch 69/100
9/9 [==============================] - ETA: 0s - loss: 0.4517 - accuracy: 0.9968 - f1_score: 0.3913 - precision: 0.2500 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8803.0000 - false_positives: 27.0000 - false_negatives: 1.0000WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00069: loss improved from 0.45733 to 0.45175, saving model to ../models\mining-detector-checkpoint.h5
9/9 [==============================] - 0s 39ms/step - loss: 0.4517 - accuracy: 0.9968 - f1_score: 0.3913 - precision: 0.2500 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8803.0000 - false_positives: 27.0000 - false_negatives: 1.0000
Epoch 70/100
9/9 [==============================] - ETA: 0s - loss: 0.4481 - accuracy: 0.9982 - f1_score: 0.5556 - precision: 0.3846 - recall: 1.0000 - true_positi

Epoch 79/100
9/9 [==============================] - ETA: 0s - loss: 0.4293 - accuracy: 0.9979 - f1_score: 0.4865 - precision: 0.3333 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8812.0000 - false_positives: 18.0000 - false_negatives: 1.0000             WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00079: loss did not improve from 0.42924
9/9 [==============================] - 0s 31ms/step - loss: 0.4293 - accuracy: 0.9979 - f1_score: 0.4865 - precision: 0.3333 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8812.0000 - false_positives: 18.0000 - false_negatives: 1.0000
Epoch 80/100
9/9 [==============================] - ETA: 0s - loss: 0.4282 - accuracy: 0.9958 - f1_score: 0.3019 - precision: 0.1860 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8795.0000 - false_

Epoch 89/100
9/9 [==============================] - ETA: 0s - loss: 0.4053 - accuracy: 0.9962 - f1_score: 0.2609 - precision: 0.1667 - recall: 0.6000 - true_positives: 6.0000 - true_negatives: 8800.0000 - false_positives: 30.0000 - false_negatives: 4.0000             WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00089: loss did not improve from 0.40268
9/9 [==============================] - 0s 31ms/step - loss: 0.4053 - accuracy: 0.9962 - f1_score: 0.2609 - precision: 0.1667 - recall: 0.6000 - true_positives: 6.0000 - true_negatives: 8800.0000 - false_positives: 30.0000 - false_negatives: 4.0000
Epoch 90/100
9/9 [==============================] - ETA: 0s - loss: 0.4007 - accuracy: 0.9979 - f1_score: 0.4865 - precision: 0.3333 - recall: 0.9000 - true_positives: 9.0000 - true_negatives: 8812.0000 - false_

Epoch 99/100
9/9 [==============================] - ETA: 0s - loss: 0.3804 - accuracy: 0.9984 - f1_score: 0.5333 - precision: 0.4000 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8818.0000 - false_positives: 12.0000 - false_negatives: 2.0000WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,f1_score,precision,recall,true_positives,true_negatives,false_positives,false_negatives,lr

Epoch 00099: loss did not improve from 0.37818
9/9 [==============================] - 0s 32ms/step - loss: 0.3804 - accuracy: 0.9984 - f1_score: 0.5333 - precision: 0.4000 - recall: 0.8000 - true_positives: 8.0000 - true_negatives: 8818.0000 - false_positives: 12.0000 - false_negatives: 2.0000
Epoch 100/100
9/9 [==============================] - ETA: 0s - loss: 0.3768 - accuracy: 0.9984 - f1_score: 0.5882 - precision: 0.4167 - recall: 1.0000 - true_positives: 10.0000 - true_negatives: 8816.0000 - false_positives: 

In [35]:
confusion_matrix(target, (model.predict([cons_info_train, cons_month, cons_day]) > 0.5).astype(int).ravel())

array([[8824,    6],
       [   2,    8]], dtype=int64)

<h2>保存模型</h2>

In [36]:
# save_model(model=model, filepath="../models/mining-dector/")

<h2>加载模型</h2>

In [37]:
# model = load_model(filepath="../models/mining-dector/", custom_objects={"add_features": add_features})

In [38]:
model.load_weights("../models/mining-detector-checkpoint.h5")

<h1>预测测试集</h1>

<h2>处理数据</h2>

<h3>处理档案数据</h3>

In [39]:
cons_info_test = pd.read_csv("../data/测试组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE
ID,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,NaN,24,1999/4/1,非临时用电,NaN
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,NaN,36,1999/3/1,非临时用电,NaN
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,NaN,120,2019/11/8 12:42:00,非临时用电,NaN
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN


In [40]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,24,1998/12/1,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,24,1999/4/1,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,36,1999/3/1,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,120,2019/11/8 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电


In [41]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [42]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [43]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now
ID,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31


In [44]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now,live_days,check_days
ID,,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773


In [45]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,live_days,check_days
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,24,非临时用电,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,36,非临时用电,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,120,非临时用电,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,120,非临时用电,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,120,非临时用电,773,773


In [46]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  15379 non-null  object 
 1   VOLT_NAME       15379 non-null  object 
 2   PRC_NAME        15379 non-null  object 
 3   CONTRACT_CAP    15379 non-null  float64
 4   RUN_CAP         15379 non-null  float64
 5   SHIFT_NO        15379 non-null  float64
 6   CHK_CYCLE       15379 non-null  int64  
 7   TMP_NAME        15379 non-null  object 
 8   live_days       15379 non-null  int64  
 9   check_days      15379 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.3+ MB


In [47]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活1_10kV(合表),PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,1,0,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1


In [48]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                15379 non-null  float64
 1   RUN_CAP                     15379 non-null  float64
 2   SHIFT_NO                    15379 non-null  float64
 3   CHK_CYCLE                   15379 non-null  int64  
 4   live_days                   15379 non-null  int64  
 5   check_days                  15379 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     15379 non-null  uint8  
 7   ELEC_TYPE_NAME_农业生产用电       15379 non-null  uint8  
 8   ELEC_TYPE_NAME_商业用电         15379 non-null  uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     15379 non-null  uint8  
 10  ELEC_TYPE_NAME_大工业用电        15379 non-null  uint8  
 11  ELEC_TYPE_NAME_学校教学和学生生活用电  15379 non-null  uint8  
 12  ELEC_TYPE_NAME_居民生活用电       15379 non-null  uint8  
 13  ELEC_TYPE_NAME_普通工

In [49]:
cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [50]:
cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [51]:
cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1


In [52]:
cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,0,0,0,0,0,1,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1


In [53]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,0,0,0,0,0,1,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1


In [54]:
cons_info_test_values = cons_info_test.values
cons_info_test_values.shape

(15379, 20)

<h3>处理月电量数据</h3>

In [55]:
elec_month_test = pd.read_csv("../data/测试组_比特币挖矿_月用电明细（20211217）.csv")
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179438260,202001,244,111,1,356
1,179438260,202002,217,108,-1,324
2,179438260,202003,280,90,0,370
3,179438260,202004,338,118,1,457
4,179438260,202005,297,120,-1,416
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


In [56]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179438260,202001,244,111,1,356
1,179438260,202002,217,108,-1,324
2,179438260,202003,280,90,0,370
3,179438260,202004,338,118,1,457
4,179438260,202005,297,120,-1,416
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]] = elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]] / elec_month_test[["pq_f", "pq_g", "pq_p", "pq_z"]].max(axis=0)
elec_month_test

In [57]:
cons_month_result_test = []
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_month_result_test.append(df.values)
cons_month_test = np.array(cons_month_result_test)
cons_month_test.shape

100%|█████████████████████████████████████████████████████████████████████████████| 15379/15379 [00:27<00:00, 554.89it/s]


(15379, 22, 4)

<h3>处理日电量数据</h3>

In [58]:
elec_day_test = pd.read_csv("../data/测试组_比特币挖矿_日用电明细（20211217）.csv")
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,179404030,2020-01-22 00:00:00,51.77,NaN,0.0,33.47,0.0,18.31
1,179404030,2020-01-23 00:00:00,73.27,NaN,0.0,51.83,0.0,21.44
2,179404030,2020-01-24 00:00:00,74.90,NaN,0.0,52.20,0.0,22.70
3,179404030,2020-01-25 00:00:00,62.73,NaN,0.0,37.27,0.0,25.45
4,179404030,2020-01-26 00:00:00,64.27,NaN,0.0,41.72,0.0,22.56
...,...,...,...,...,...,...,...,...
1660582,2880712108,2021-10-05 00:00:00,44.85,NaN,0.0,22.14,0.0,22.71
1660583,2880712108,2021-10-06 00:00:00,28.89,NaN,0.0,14.94,0.0,13.95
1660584,2880712108,2021-10-07 00:00:00,51.10,NaN,0.0,27.27,0.0,23.84
1660585,2880712108,2021-10-08 00:00:00,17.96,NaN,0.0,7.19,0.0,10.76


elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] = elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] / elec_day_test[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]].max(axis=0)
elec_day_test

In [59]:
dates = pd.to_datetime(elec_day_test["rq"].unique()).sort_values()

In [60]:
cons_day_result_test = []
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id].drop(columns=["id"]).sort_values("rq")
    df["rq"] = pd.to_datetime(df["rq"])
    df = df.set_index("rq").reindex(dates)
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_day_result_test.append(df.values)
cons_day_test = np.array(cons_day_result_test)
cons_day_test.shape

100%|█████████████████████████████████████████████████████████████████████████████| 15379/15379 [01:12<00:00, 212.15it/s]


(15379, 108, 6)

<h2>预测数据</h2>

In [61]:
y_pred = model.predict([cons_info_test_values, cons_month_test, cons_day_test])
y_pred

array([[0.30003577],
       [0.3047074 ],
       [0.30015182],
       ...,
       [0.31807232],
       [0.29941654],
       [0.29775432]], dtype=float32)

In [62]:
cons_info_test["label"] = (y_pred > 0.5).astype(int)
cons_info_test.index.name = "id"
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电,label
id,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,1,0,0,1,1,0
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,1,1,0
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,1,1,0


In [63]:
cons_info_test[["label"]].to_csv("../data/result20220221-1.csv", index=True)